In [1]:
import os
import base64
from typing import List, Dict, Union
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
# OPENAI_API_KEY가 .env 파일에 설정되어 있어야 합니다.
load_dotenv()

True

In [2]:
# 1. 추출하고자 하는 정보에 대한 스키마 정의
response_schemas = [
    ResponseSchema(name="product_name", description="상품의 전체 이름"),
    ResponseSchema(name="brand_name", description="상품의 브랜드 이름"),
]

# 2. 스키마를 기반으로 출력 파서 생성
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# 3. 프롬프트 템플릿에 파서의 형식 지시 포함
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"product_name": string  // 상품의 전체 이름
	"brand_name": string  // 상품의 브랜드 이름
}
```


In [ ]:
prompt = PromptTemplate(
    template="""주어진 내용에서 사용자가 요청한 정보를 추출해줘.
내용:
{content}

{format_instructions}""",
    input_variables=["content"],
    partial_variables={"format_instructions": format_instructions},
)

# 4. 분석할 원본 텍스트
content = """
아래 내용은 '유그린(Ugreen)' 이라는 브랜드의 '300W GaN PD PPS QC 5포트 초고속 충전기 CD333' 이라는 상품의 상세 정보 테이블입니다.

기본 정보: 상품번호, 상품상태, 제조사, 브랜드, 모델명, 이벤트, 원산지, 제조일자 등의 일반적인 상품 정보

상품 유형: 충전기의 형태(콘센트형), 품목(스마트폰 충전기) 등 구체적인 분류 정보

성능 및 규격: 충전기 출력(300W), 단자(Type-C, USB 포트 각 1개), 충전기능(고속충전, 동시충전, PPS, USB-PD3.1), 부가기능(GaN 충전), 안전기능(과전압보호, 과전류보호), 크기(5.3x11.1x6.5cm), 케이블 포함 여부 및 길이 등 기술적인 사양
"""

In [ ]:
# 5. 모델 인스턴스 생성
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# 6. 프롬프트와 모델, 출력 파서를 연결하여 체인 생성 및 실행
chain = prompt | model | output_parser
result = chain.invoke({"content": content})

# 7. 추출 결과 확인
print("추출된 브랜드명:", result["brand_name"])
print("추출된 제품명:", result["product_name"])

추출된 브랜드명: 유그린(Ugreen)
추출된 제품명: 300W GaN PD PPS QC 5포트 초고속 충전기 CD333
